In [7]:
from scraper import Scraper
from splitter import Splitter
from io_helper import IOHelper
from server import ServerHelper

scraper = Scraper()
splitter = Splitter()
io_helper = IOHelper()
server = ServerHelper()

2024-10-20 13:40:15,332 - WARNING - langchain_community.utils.user_agent - USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
server.health()

{'response': 'ok'}

In [5]:
files = list(io_helper.iterate_over_files("files"))
ingestion_resp = server.ingest_files(files)

Ingested: files\2311.09476v2.pdf ...


In [12]:
server.documents_overview()

[]

In [9]:
chunks = server.document_chunks(doc_id)

In [9]:
rag_rsp = server.rag("What is ARES?")

In [10]:
print(rag_rsp['completion']['choices'][0]['message']['content'])

I don't see any context provided. Please provide the context, and I'll be happy to help answer the query "What is ARES?" using line item references as needed.
